# 데이터를 불러온 뒤 학습데이터와 테스트데이터로 분류작업을 실시.

In [3]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

OSError: [WinError 126] 지정된 모듈을 찾을 수 없습니다. Error loading "C:\Users\bigca\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [4]:
import numpy as np
import pandas as pd

train_ft = pd.read_excel("train_data.xlsx") # 학습데이터

train_ft.shape

(4983, 10)

In [7]:
train_ft.drop(columns = ['Unnamed: 9'],inplace = True)

train_ft.columns

Index(['Review', '총장 길다', '총장 짧다', '어깨너비 크다', '어깨너비 작다', '가슴단면 크다', '가슴단면 작다',
       '소매길이 길다', '소매길이 짧다', 'Unnamed: 10'],
      dtype='object')

In [7]:
!pip install iterative-stratification

In [8]:
# 학습데이터와 테스트데이터로 구분 작업 시작
x,y = train_ft['Review'].to_numpy(), train_ft[['총장 길다', '총장 짧다', '어깨너비 크다', '어깨너비 작다', '가슴단면 크다', '가슴단면 작다', '소매길이 길다','소매길이 짧다']].replace(-1,1).to_numpy()

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

msss = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in msss.split(x, y):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    break

tmp_1 = pd.DataFrame(x_train,columns = ['Review'])
tmp_2 = pd.DataFrame(y_train,columns = ['총장 길다', '총장 짧다', '어깨너비 크다', '어깨너비 작다', '가슴단면 크다', '가슴단면 작다', '소매길이 길다','소매길이 짧다'])

train_ft = pd.concat([tmp_1,tmp_2],axis = 1)

tmp_1 = pd.DataFrame(x_test,columns = ['Review'])
tmp_2 = pd.DataFrame(y_test,columns = ['총장 길다', '총장 짧다', '어깨너비 크다', '어깨너비 작다', '가슴단면 크다', '가슴단면 작다', '소매길이 길다','소매길이 짧다'])

test_ft = pd.concat([tmp_1,tmp_2],axis = 1)

In [9]:
train_ft.shape,test_ft.shape

((3987, 9), (996, 9))

In [10]:
# 딥러닝 학슴을 위해서 array의 형태로 변환 작업이 행해지는 것이다.
# 리뷰만 뽑아준다.

train_arr = train_ft['Review'].to_numpy()
test_arr = test_ft['Review'].to_numpy()

train_arr.shape, test_arr.shape

((3987,), (996,))

In [11]:
# 딥러닝 학습을 위해서 target에 대해 array의 형태로 변환작업이 행해지는 것이다.

target = train_ft.iloc[:,1:].to_numpy().copy()
target_test = test_ft.iloc[:,1:].to_numpy().copy()

target.shape, target_test.shape

((3987, 8), (996, 8))

# 학습 실시

In [12]:
model_name = 'kykim/bert-kor-base'

In [13]:
from transformers import AutoTokenizer, AutoModel

In [14]:
# 사전학습 모델을 불러온다.

model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [15]:
# 사전학습모델에 존재하는 tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.model_max_length

tokenizer_config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

512

In [16]:
class ReviewDataset(torch.utils.data.Dataset):
  def __init__(self,tokenizer, x,y = None):
    self.tokenizer = tokenizer
    self.x = x
    self.y = y

  def __len__(self):
    return len(self.x)

  def __getitem__(self,idx):
    item = {}
    item['x'] = self.get_tokenizer(self.x[idx])
    if self.y is not None:
      item['y'] = torch.Tensor(self.y[idx])
    return item

  def get_tokenizer(self,text):
    x = self.tokenizer(text,padding = 'max_length',truncation = True)
    for k,v in x.items():
      x[k] = torch.tensor(v)
    return x

In [17]:
dt = ReviewDataset(tokenizer,train_arr,target)
dl = torch.utils.data.DataLoader(dt,batch_size = 2, shuffle = False)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[    2, 15219, 24151,  ...,     0,     0,     0],
         [    2, 21752, 17364,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])},
 'y': tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]])}

In [18]:
class Net(torch.nn.Module):
  def __init__(self, model_name):
    super().__init__()
    self.pre_model = AutoModel.from_pretrained(model_name)
    self.fc_out = torch.nn.Linear(self.pre_model.config.hidden_size,8)

  def forward(self,x):
    x = self.pre_model(**x)
    return self.fc_out(x[1])

In [19]:
model = Net(model_name)
model(batch['x'])

tensor([[ 0.2185,  0.3710, -0.3428,  0.4489, -0.5170, -0.7328,  0.3718, -0.6209],
        [ 0.1576,  0.3182, -0.5115,  0.5600, -0.4509, -0.7701,  0.3934, -0.7514]],
       grad_fn=<AddmmBackward0>)

In [20]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
  epoch_loss = 0
  model.train()
  for batch in tqdm(dataloader):
    pred = model(batch['x'].to(device))
    loss = loss_fn(pred,batch['y'].to(device))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

  epoch_loss /= len(dataloader)

  return epoch_loss

In [21]:
@torch.no_grad()
def test_loop(dataloader,model,loss_fn,device):
  epoch_loss = 0
  pred_list = []
  act_func = torch.nn.Sigmoid()
  model.eval()
  for batch in tqdm(dataloader):
    pred = model(batch['x'].to(device))
    if batch.get('y') is not None:
      loss = loss_fn(pred, batch['y'].to(device))
      epoch_loss += loss.item()

    pred = act_func(pred)
    pred = pred.to('cpu').numpy()
    pred_list.append(pred)

  epoch_loss /= len(dataloader)
  pred = np.concatenate(pred_list)
  return epoch_loss,pred

In [22]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

n_splits = 5
cv = KFold(n_splits, shuffle=True, random_state=42)

batch_size = 8 # 배치 사이즈
loss_fn = torch.nn.BCEWithLogitsLoss() # 손실 객체
epochs = 100 # 최대 가능한 에폭수

In [ ]:
is_holdout = True
reset_seeds(42) # 재현을 위해 시드고정
best_score_list = []
for i, (tri, vai) in enumerate( cv.split(train_arr) ):
    # 학습용 데이터로더 객체
    train_dt = ReviewDataset(tokenizer, train_arr[tri], target[tri])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)

    # 검증용 데이터로더 객체
    valid_dt = ReviewDataset(tokenizer, train_arr[vai], target[vai])
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size, shuffle=False)

    # 모델 객체와 옵티마이저 객체 생성
    model = Net(model_name).to(device)
    optimizer = torch.optim.Adam( model.parameters(), lr=2e-5 )

    best_score = 0 # 현재 최고 점수
    patience = 0 # 조기 종료 조건을 주기 위한 변수
    for epoch in range(epochs):
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
        valid_loss, pred = test_loop(valid_dl, model, loss_fn, device)

        pred = (pred > 0.5).astype(int) # 이진분류 문제에서 클래스 번호 결정
        score = f1_score(target[vai], pred,average = None).mean()

        print(train_loss, valid_loss, score)
        if score > best_score:
            best_score = score # 최고 점수 업데이트
            patience = 0
            torch.save(model.state_dict(), f"model_{i}.pth") # 최고 점수 모델 가중치 저장

        patience += 1
        if patience == 5:
            break

    print(f"{i}번째 폴드 최고 정확도: {best_score}")
    best_score_list.append(best_score)

    if is_holdout:
        break

  0%|          | 0/399 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

0.18280952338848197 0.1160501369740814 0.42293974637939424


  0%|          | 0/399 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

0.009151413214914058 0.14489192415669094 0.8221542011841922


  0%|          | 0/399 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

0.017409336354707795 0.09174042122089304 0.7819353625471928
0번째 폴드 최고 정확도: 0.8908823727624875


# 테스트 데이터에 대해서 예측하고 검증 실행

In [23]:
test_dt = ReviewDataset(tokenizer,test_arr)
test_dl = torch.utils.data.DataLoader(test_dt, batch_size=2, shuffle=False)

In [24]:
model = Net(model_name).to(device)
state_dict = torch.load(f'{DATA_PATH}model_0.pth')
model.load_state_dict(state_dict)

_, pred = test_loop(test_dl,model,loss_fn,device)

pred

  0%|          | 0/498 [00:00<?, ?it/s]

array([[1.0206958e-03, 5.8897049e-04, 8.1789505e-04, ..., 2.5863296e-04,
        7.9982483e-04, 3.8410971e-04],
       [9.0135010e-03, 9.9233538e-01, 9.5987618e-03, ..., 9.5393896e-01,
        5.0945180e-03, 9.5563447e-01],
       [8.2268374e-04, 5.4203020e-04, 7.1496388e-04, ..., 2.8214944e-04,
        7.0493517e-04, 3.7449828e-04],
       ...,
       [7.5821503e-04, 5.1792269e-04, 7.4750773e-04, ..., 2.9142422e-04,
        7.1898219e-04, 4.0038393e-04],
       [6.4316863e-04, 6.0836144e-04, 6.6372257e-04, ..., 3.4047072e-04,
        6.8422384e-04, 4.1413741e-04],
       [1.3333999e-03, 3.8185008e-04, 1.1827101e-03, ..., 2.2507658e-04,
        9.0813654e-04, 3.5995722e-04]], dtype=float32)

In [25]:
pred = (pred > 0.5).astype(int)

In [26]:
pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [27]:
score = f1_score(target_test, pred,average = 'macro')
score
# 0.834513944131179 모델의 성능은 괜찮음

0.834513944131179

# 완성된 모델을 바탕으로 새로운 리뷰에 대한 예측 작업을 진행한다.

In [28]:
# 기존이 리뷰를 불러온다.

test = pd.read_excel(f'{DATA_PATH}train_tmp.xlsx')
test.head()

,Nickname,Product_Name,Review,Gender,Height,weight,Size,Length,Sholder,Chest,Sleeve,Length_x,Sholder_x,Chest_x,Sleeve_x,Length_y,Sholder_y,Chest_y,Sleeve_y
0,LV.4 뮈치고싶다,[코싸] koxa symbol mtm all 4color 맨투맨,오버핏은 아니구 정핏입니다\n생각보가 깔끔하고 안에 입기 좋아요~,남성,174,62,L,72.0,48.0,55.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LV.7 후희,[코싸] koxa symbol mtm all 4color 맨투맨,선명하고\n길이도 품도\n다 잘 맞네요\n잘 입고다닙다.,남성,170,70,L,72.0,48.0,55.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LV.5 순천간판,[코싸] koxa symbol mtm all 4color 맨투맨,정핏으로 나왔고요. 무난하니 이쁘네요. 추천해요,남성,178,61,L,72.0,48.0,55.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LV.4 뮈치고싶다,[코싸] koxa symbol mtm all 4color 맨투맨,오버핏은 아니구 정핏입니다\n생각보가 깔끔하고 안에 입기 좋아요~,남성,174,62,L,72.0,48.0,55.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LV.7 후희,[코싸] koxa symbol mtm all 4color 맨투맨,선명하고\n길이도 품도\n다 잘 맞네요\n잘 입고다닙다.,남성,170,70,L,72.0,48.0,55.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# 모델의 가중치를 받아온 뒤 예측을 진행
@torch.no_grad()
def test_loop_classification(dataloader, model, device):
    pred_list = []
    act_func = torch.nn.Sigmoid()
    model.eval() # 평가 모드
    for batch in tqdm(dataloader):
        pred = model( batch["x"].to(device) )
        pred = act_func(pred) # logit 값을 확률로 변환
        pred = pred.to("cpu").numpy() # cpu 이동후 ndarray 로변환
        pred_list.append(pred)

    pred = np.concatenate(pred_list)
    return pred

In [ ]:
# 리뷰별로 초장, 어깨너비, 가슴단면, 소매길이에 대해 느끼는 점에 대한 예측 작업을 진행하는 함수를 만듬.
# 가중치가 여러개 존재하는 경우

def Review_classification(review):
    if isinstance(review,pd.Series): # review가 Series의 형태와 그렇지 않은 경우로 나뉨. 그렇지 않은 경우에는 list의 형식을 전달
        test_arr = review.to_numpy()
    else:
        test_arr = pd.Series(review).to_numpy()

    test_dt = ReviewDataset(tokenizer,test_arr)
    test_dl = torch.utils.data.DataLoader(test_dt,batch_size = 8,shuffle = False)

    pred_list = []

    model = Net(model_name).to(device)
    state_dict = torch.load(f'{DATA_PATH}model_0.pth')
    model.load_state_dict(state_dict)

    pred = test_loop_classification(test_dl, model, device)
    pred_list.append(pred)
    pred = np.mean(pred_list, axis=0)
    pred = (pred > 0.5).astype(int)

    pred = pd.DataFrame(pred,columns = ['총장 길다','총장 짧다','어깨너비 크다','어깨너비 작다','가슴단면 크다','가슴단면 작다','소매길이 길다','소매길이 짧다'])
    review = pd.DataFrame(review).reset_index(drop = True)
    pred = pd.concat([review,pred],axis = 1)

    pred['총장'] = np.where((pred['총장 길다'] == 1) & (pred['총장 짧다'] == 0), 1,
                   np.where((pred['총장 길다'] == 0) & (pred['총장 짧다'] == 1), -1,
                   np.where((pred['총장 길다'] == 0) & (pred['총장 짧다'] == 0), 0,None)))

    pred['어깨너비'] = np.where((pred['어깨너비 크다'] == 1) & (pred['어깨너비 작다'] == 0), 1,
                       np.where((pred['어깨너비 크다'] == 0) & (pred['어깨너비 작다'] == 1), -1,
                       np.where((pred['어깨너비 크다'] == 0) & (pred['어깨너비 작다'] == 0), 0,None)))

    pred['가슴단면'] = np.where((pred['가슴단면 크다'] == 1) & (pred['가슴단면 작다'] == 0), 1,
                       np.where((pred['가슴단면 크다'] == 0) & (pred['가슴단면 작다'] == 1), -1,
                       np.where((pred['가슴단면 크다'] == 0) & (pred['가슴단면 작다'] == 0), 0,None)))

    pred['소매길이'] = np.where((pred['소매길이 길다'] == 1) & (pred['소매길이 짧다'] == 0), 1,
                       np.where((pred['소매길이 길다'] == 0) & (pred['소매길이 짧다'] == 1), -1,
                       np.where((pred['소매길이 길다'] == 0) & (pred['소매길이 짧다'] == 0), 0,None)))

    pred = pred.rename(columns = {0:'Review'})

    pred.drop(columns = ['총장 길다','총장 짧다','어깨너비 크다','어깨너비 작다','가슴단면 크다','가슴단면 작다','소매길이 길다','소매길이 짧다'],inplace = True)

    return pred

In [30]:
# 리뷰별로 초장, 어깨너비, 가슴단면, 소매길이에 대해 느끼는 점에 대한 예측 작업을 진행하는 함수를 만듬.
# 가중치가 하나만 존재하는 경우

def Review_classification(review):
    if isinstance(review,pd.Series): # review가 Series의 형태와 그렇지 않은 경우로 나뉨. 그렇지 않은 경우에는 list의 형식을 전달
        test_arr = review.to_numpy()
    else:
        test_arr = pd.Series(review).to_numpy()

    test_dt = ReviewDataset(tokenizer,test_arr)
    test_dl = torch.utils.data.DataLoader(test_dt,batch_size = 8,shuffle = False)

    model = Net(model_name).to(device)
    state_dict = torch.load(f'{DATA_PATH}model_0.pth')
    model.load_state_dict(state_dict)

    pred = test_loop_classification(test_dl, model, device)
    pred = (pred > 0.5).astype(int)

    pred = pd.DataFrame(pred,columns = ['총장 길다','총장 짧다','어깨너비 크다','어깨너비 작다','가슴단면 크다','가슴단면 작다','소매길이 길다','소매길이 짧다'])
    review = pd.DataFrame(review).reset_index(drop = True)
    pred = pd.concat([review,pred],axis = 1)

    pred['총장'] = np.where((pred['총장 길다'] == 1) & (pred['총장 짧다'] == 0), 1,
                   np.where((pred['총장 길다'] == 0) & (pred['총장 짧다'] == 1), -1,
                   np.where((pred['총장 길다'] == 0) & (pred['총장 짧다'] == 0), 0,None)))

    pred['어깨너비'] = np.where((pred['어깨너비 크다'] == 1) & (pred['어깨너비 작다'] == 0), 1,
                       np.where((pred['어깨너비 크다'] == 0) & (pred['어깨너비 작다'] == 1), -1,
                       np.where((pred['어깨너비 크다'] == 0) & (pred['어깨너비 작다'] == 0), 0,None)))

    pred['가슴단면'] = np.where((pred['가슴단면 크다'] == 1) & (pred['가슴단면 작다'] == 0), 1,
                       np.where((pred['가슴단면 크다'] == 0) & (pred['가슴단면 작다'] == 1), -1,
                       np.where((pred['가슴단면 크다'] == 0) & (pred['가슴단면 작다'] == 0), 0,None)))

    pred['소매길이'] = np.where((pred['소매길이 길다'] == 1) & (pred['소매길이 짧다'] == 0), 1,
                       np.where((pred['소매길이 길다'] == 0) & (pred['소매길이 짧다'] == 1), -1,
                       np.where((pred['소매길이 길다'] == 0) & (pred['소매길이 짧다'] == 0), 0,None)))

    pred = pred.rename(columns = {0:'Review'})

    pred.drop(columns = ['총장 길다','총장 짧다','어깨너비 크다','어깨너비 작다','가슴단면 크다','가슴단면 작다','소매길이 길다','소매길이 짧다'],inplace = True)

    pred['총장'] = pred['총장'].replace([-1,0,1],['짧다','정핏','길다'])
    pred['어깨너비'] = pred['어깨너비'].replace([-1,0,1],['작다','정핏','크다'])
    pred['가슴단면'] = pred['가슴단면'].replace([-1,0,1],['작다','정핏','크다'])
    pred['소매길이'] = pred['소매길이'].replace([-1,0,1],['짧다','정핏','길다'])

    return pred

In [31]:
pred = Review_classification(['가슴 폭이 너무 좁아요'])
pred

  0%|          | 0/1 [00:00<?, ?it/s]

,Review,총장,어깨너비,가슴단면,소매길이
0,가슴 폭이 너무 좁아요,정핏,정핏,작다,정핏


# 예측된 파일을 엑셀로 저장

In [ ]:
pred_tmp = pd.concat([pred,test.iloc[:,3:11]],axis = 1)
pred_tmp.shape

(42650, 17)

In [ ]:
pred_tmp.to_excel('review_42650.xlsx',index = False)

# 모델링 된 것을 바탕으로 시각화 자료 제시

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
review_ft = pd.read_excel('review_ft.xlsx')
review_ft.shape

(42634, 14)

In [ ]:
review_ft['총장'] = review_ft['총장'].replace([-1,0,1],['짧다','정핏','길다'])
review_ft['어깨너비'] = review_ft['어깨너비'].replace([-1,0,1],['작다','정핏','크다'])
review_ft['가슴단면'] = review_ft['가슴단면'].replace([-1,0,1],['작다','정핏','크다'])
review_ft['소매길이'] = review_ft['소매길이'].replace([-1,0,1],['짧다','정핏','길다'])

review_ft.head()

,Product_Name,Review,Gender,Height,weight,Size,Length,Sholder,Chest,Sleeve,총장,어깨너비,가슴단면,소매길이
0,[코싸] koxa symbol mtm all 4color 맨투맨,오버핏은 아니구 정핏입니다\n생각보가 깔끔하고 안에 입기 좋아요~,남성,174,62,L,72.0,48.0,55.0,68.0,정핏,정핏,정핏,정핏
1,[코싸] koxa symbol mtm all 4color 맨투맨,선명하고\n길이도 품도\n다 잘 맞네요\n잘 입고다닙다.,남성,170,70,L,72.0,48.0,55.0,68.0,정핏,정핏,정핏,정핏
2,[코싸] koxa symbol mtm all 4color 맨투맨,정핏으로 나왔고요. 무난하니 이쁘네요. 추천해요,남성,178,61,L,72.0,48.0,55.0,68.0,정핏,정핏,정핏,정핏
3,[코싸] koxa symbol mtm all 4color 맨투맨,오버핏은 아니구 정핏입니다\n생각보가 깔끔하고 안에 입기 좋아요~,남성,174,62,L,72.0,48.0,55.0,68.0,정핏,정핏,정핏,정핏
4,[코싸] koxa symbol mtm all 4color 맨투맨,선명하고\n길이도 품도\n다 잘 맞네요\n잘 입고다닙다.,남성,170,70,L,72.0,48.0,55.0,68.0,정핏,정핏,정핏,정핏


In [5]:
product_list = list(review_ft['Product_Name'].unique())
product_list

['[코싸] koxa symbol mtm all 4color 맨투맨',
 'EMBROIDERY UNIVERSITY DAN CREWNECK NAVY',
 '우먼즈 세미 오버사이즈 스웨트셔츠 [미디엄 그레이]',
 '피치스킨 오버핏 맨투맨 MMT101 [PASTELBLUE]',
 '오버사이즈 폴라 플리스 스웨트셔츠 [블랙]',
 '엑스트라 오버사이즈 스웨트셔츠 [블랙]',
 '나비고양이 특양면 맨투맨_B1',
 '헤비 코튼 오버 럭비 맨투맨_Black',
 'RETRIEVER FRIENDS CREWNECK [CHARCOAL]',
 'V.S.C SWEAT (WISCONSIN)_RED',
 '기모 스웨트셔츠 [미디엄 그레이]',
 '980G 피그먼트 맨투맨-샌드 베이지-',
 '우먼즈 세미 오버사이즈 스웨트셔츠 [블랙]',
 'NYC LOCATION SWEATSHIRT (LIGHT BLUE) [LRSFCTM302M]',
 '[SET] (긴바지 OR 반바지 선택) 24H 트랙 브이넥 맨투맨 셋업_네이비',
 '컬러 블록 맨투맨 아이보리',
 '오버핏 블룸 스웨트 셔츠-블랙',
 '오버사이즈 하프 집업 스웨트셔츠 [미디엄 그레이]',
 '하프 집업 스웻셔츠 쇼츠 세트',
 '우먼즈 크롭 베어프렌즈 엠보자수 맨투맨 그린(FCB3TS355W)',
 '[UNISEX]플라워도트 자수 맨투맨_2color',
 '스웨트셔츠 [그린]',
 '[SET] (긴바지 OR 반바지 선택) CGP 헤리티지 스트라이프 럭비 맨투맨 셋업_네이비',
 '헤비웨이트 오버사이즈 스웨트셔츠 [다크 브라운]',
 '90S 트랙 스웨트셔츠_네이비',
 '스웨트셔츠 [크림]',
 '오버핏 아치 로고 스웨트 셔츠-라이트 그린',
 'CLASSIC LOGO SWEATSHIRT oatmeal',
 'PRDA CREWNECK SWEATSHIRT HEATHER GRAY',
 '스웨트셔츠 [베이지]',
 '[2PACK] 블루밍 맨투맨 6종 JJMT7204',
 '콘트라스트 스티치 스웨트셔츠 [블랙]',
 '루바토 브이넥

In [8]:
product_list[40]

'우먼즈 크롭 르 리스 발포 엠보자수 맨투맨 네이비(FCB4TS360W)'

In [ ]:
review_ft[review_ft['Product_Name'] == product_list[40]]['총장'].value_counts()
# 40번째 상품을 예시로 크롤링 작업을 진행해본다.

총장
 0.0    182
-1.0     68
 1.0     32
Name: count, dtype: int64

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

def visual_fit(review_ft,Product_name):
  mask = review_ft['Product_Name'] == Product_name
  tmp_df = review_ft[mask].copy()

  # 총장 시각화
  plt.figure(figsize = (10,8))
  plt.subplot(2,2,1)
  tmp_df['총장'].value_counts().plot(kind = 'pie', autopct = '%1.1f%%')
  plt.title('총장')

  # 어깨너비 시각화
  plt.subplot(2,2,2)
  tmp_df['어깨너비'].value_counts().plot(kind = 'pie',autopct = '%1.1f%%')
  plt.title('어깨너비')

  # 가슴단면 시각화
  plt.subplot(2,2,3)
  tmp_df['가슴단면'].value_counts().plot(kind = 'pie',autopct = '%1.1f%%')
  plt.title('가슴단면')

  plt.subplot(2,2,4)
  tmp_df['소매길이'].value_counts().plot(kind = 'pie',autopct = '%1.1f%%')
  plt.title('소매길이')

  plt.tight_layout()
  plt.show()

In [9]:
product_list[89]

'1960 sweatshirts khaki'

In [2]:
visual_fit(review_ft,product_list[85])
# 78,80,81,85,89
# 98,85,81,78

NameError: name 'visual_fit' is not defined

# LDA 진행

In [1]:
pip install kiwipiepy


     ---------------------------------------- 0.0/34.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/34.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/34.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/34.7 MB ? eta -:--:--
     --------------------------------------- 0.0/34.7 MB 325.1 kB/s eta 0:01:47
     --------------------------------------- 0.0/34.7 MB 279.3 kB/s eta 0:02:05
     --------------------------------------- 0.1/34.7 MB 521.8 kB/s eta 0:01:07
     --------------------------------------- 0.1/34.7 MB 599.1 kB/s eta 0:00:58
     --------------------------------------- 0.1/34.7 MB 566.5 kB/s eta 0:01:02
     --------------------------------------- 0.2/34.7 MB 581.0 kB/s eta 0:01:00
     --------------------------------------- 0.2/34.7 MB 565.6 kB/s eta 0:01:02
     --------------------------------------- 0.3/34.7 MB 628.5 kB/s eta 0:00:55
     --------------------------------------- 0.3/34.7 MB 678.0 kB/s eta

In [14]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [8]:
df = pd.read_excel('review_ft_2.xlsx')
df.head()

,Product_Name,Review,Gender,Height,weight,Size,Length,Sholder,Chest,Sleeve,총장,어깨너비,가슴단면,소매길이
0,[코싸] koxa symbol mtm all 4color 맨투맨,오버핏은 아니구 정핏입니다\n생각보가 깔끔하고 안에 입기 좋아요~,남성,174,62,L,72.0,48.0,55.0,68.0,0.0,0.0,0,0.0
1,[코싸] koxa symbol mtm all 4color 맨투맨,선명하고\n길이도 품도\n다 잘 맞네요\n잘 입고다닙다.,남성,170,70,L,72.0,48.0,55.0,68.0,0.0,0.0,0,0.0
2,[코싸] koxa symbol mtm all 4color 맨투맨,정핏으로 나왔고요. 무난하니 이쁘네요. 추천해요,남성,178,61,L,72.0,48.0,55.0,68.0,0.0,0.0,0,0.0
3,[코싸] koxa symbol mtm all 4color 맨투맨,오버핏은 아니구 정핏입니다\n생각보가 깔끔하고 안에 입기 좋아요~,남성,174,62,L,72.0,48.0,55.0,68.0,0.0,0.0,0,0.0
4,[코싸] koxa symbol mtm all 4color 맨투맨,선명하고\n길이도 품도\n다 잘 맞네요\n잘 입고다닙다.,남성,170,70,L,72.0,48.0,55.0,68.0,0.0,0.0,0,0.0


In [9]:
# 형태소 분석기 불러오기 

from kiwipiepy import Kiwi

kiwi = Kiwi()

In [10]:
# 불용어

from kiwipiepy.utils import Stopwords
stopwords = Stopwords()
stopwords.stopwords # 불용어 리스트

{('ᆫ', 'ETM'),
 ('ᆫ', 'JX'),
 ('ᆫ다', 'EF'),
 ('ᆯ', 'ETM'),
 ('가', 'JKS'),
 ('같', 'VA'),
 ('것', 'NNB'),
 ('게', 'EC'),
 ('겠', 'EP'),
 ('고', 'EC'),
 ('고', 'JKQ'),
 ('과', 'JC'),
 ('과', 'JKB'),
 ('그', 'MM'),
 ('그', 'NP'),
 ('기', 'ETN'),
 ('까지', 'JX'),
 ('나', 'NP'),
 ('년', 'NNB'),
 ('는', 'ETM'),
 ('는', 'JX'),
 ('다', 'EC'),
 ('다', 'EF'),
 ('다고', 'EC'),
 ('다는', 'ETM'),
 ('대하', 'VV'),
 ('더', 'MAG'),
 ('던', 'ETM'),
 ('도', 'JX'),
 ('되', 'VV'),
 ('되', 'XSV'),
 ('들', 'XSN'),
 ('등', 'NNB'),
 ('따르', 'VV'),
 ('때', 'NNG'),
 ('때문', 'NNB'),
 ('라', 'EC'),
 ('라는', 'ETM'),
 ('로', 'JKB'),
 ('를', 'JKO'),
 ('만', 'JX'),
 ('만', 'NR'),
 ('말', 'NNG'),
 ('며', 'EC'),
 ('면', 'EC'),
 ('면서', 'EC'),
 ('명', 'NNB'),
 ('받', 'VV'),
 ('보', 'VV'),
 ('부터', 'JX'),
 ('사람', 'NNG'),
 ('성', 'XSN'),
 ('수', 'NNB'),
 ('아니', 'VCN'),
 ('않', 'VX'),
 ('어', 'EC'),
 ('어', 'EF'),
 ('어서', 'EC'),
 ('어야', 'EC'),
 ('없', 'VA'),
 ('었', 'EP'),
 ('에', 'JKB'),
 ('에게', 'JKB'),
 ('에서', 'JKB'),
 ('와', 'JC'),
 ('와', 'JKB'),
 ('우리', 'NP'),
 ('원', 'NNB'),


In [15]:
# 형태소가 NV인 것만을 담는다. 

result = kiwi.tokenize(df["Review"], stopwords=stopwords) # stopwords=stopwords 는 불용어 제거할 경우
df_list = []

for lst in tqdm(result, total=df.shape[0]):
    tmp = [ token.form for token in lst if token.tag[0] in "NV" ]
    df_list.append(tmp)

  0%|          | 0/42650 [00:00<?, ?it/s]

In [17]:
pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
    --------------------------------------- 0.3/24.0 MB 9.9 MB/s eta 0:00:03
   - -------------------------------------- 0.6/24.0 MB 8.1 MB/s eta 0:00:03
   - -------------------------------------- 1.0/24.0 MB 9.5 MB/s eta 0:00:03
   -- ------------------------------------- 1.6/24.0 MB 9.2 MB/s eta 0:00:03
   --- ------------------------------------ 2.1/24.0 MB 9.5 MB/s eta 0:00:03
   ---- ----------------------------------- 2.6/24.0 MB 9.7 MB/s eta 0:00:03
   ----- ---------------------------------- 3.1/24.0 MB 10.0 MB/s eta 0:00:03
   ----- ---------------------------------- 3.6/24.0 MB 10.0 MB/s eta 0:00:03
   ------ --------------------------------- 4.1/24.0 MB 10.1 MB/s eta 0:00:02
   ------- -------------------------------- 4.4/24.0 MB 9.9 MB/s eta 0:00:02
   -------- ------------------------------- 5.0/24.0 MB 10.1 MB/s eta 0:00:02
   --------- ------------------------------ 5.6/24.0 MB 10.5 MB/s eta 0:00:02
 

In [18]:
# 단어사전을 불러오기 위한 라이브러리

from gensim.corpora.dictionary import Dictionary
dict_obj = Dictionary(df_list) # 단어 사전 객체가 반환된다 라고 생각해도 됨!
dict_obj

In [19]:
dict_obj.filter_extremes(10)

In [20]:
# 단어사전과 빈도수로 저장을 해준다. 

corpus = []
for d in df_list:
    bow = dict_obj.doc2bow(d) # [(단어번호, 빈도수), (단어번호, 빈도수) , (단어번호, 빈도수) ....]
    corpus.append(bow)
len(corpus)

42650

In [21]:
from gensim.models.ldamodel import LdaModel

In [22]:
!pip install pyLDAvis

  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 60.6/60.6 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ----- ---------------------------------- 0.4/2.6 MB 7.8 MB/s eta 0:00:01
   ----------- ---------------------------- 0.8/2.6 MB 8.1 MB/s eta 0:00:01
   ------------------ --------------------- 1.2/2.6 MB 8.6 MB/s eta 0:00:01
   ------------------------- -------------- 1.7/2.6 MB 8.9 MB/s eta 0:00:01
   -------------------------------- ------- 2.1/2.6 MB 9.0 MB/s eta 0:00:01
   ------------------------------------ --- 2.3/2.6 MB 8.3 MB/s eta 0:00:01
   ------------------------------------- -- 2.5/2.6 MB 7.9 MB/s eta 0:00:01
   -------------------------------

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 3.3.2 requires scipy<=1.11.4,>=1.6.1, but you have scipy 1.13.1 which is incompatible.


In [23]:
model = LdaModel(corpus, 20, dict_obj, passes = 10,random_state=42)

In [24]:
import pyLDAvis.gensim

In [25]:
pyLDAvis.enable_notebook()

In [26]:
p = pyLDAvis.gensim.prepare(model, corpus, dict_obj)

In [27]:
pyLDAvis.display(p)